# ChatBox 组件解释

这个笔记本解释了用 React 实现的 `ChatBox` 组件。该组件允许用户向聊天机器人发送消息并接收响应。

## 导入所需库

首先，我们导入必要的库和 CSS 文件。

In [ ]:
import React, { useState } from 'react';
import axios from 'axios';
import './ChatBox.css';

- `import React, { useState } from 'react';`：从 `react` 库中导入 `React` 和 `useState` 钩子。
- `import axios from 'axios';`：导入 `axios` 库，用于发送 HTTP 请求。
- `import './ChatBox.css';`：导入 `ChatBox.css` 文件，用于组件的样式。

## ChatBox 组件定义

`ChatBox` 组件被定义为一个函数组件。

In [ ]:
const ChatBox = () => {
    const [messages, setMessages] = useState([]);
    const [input, setInput] = useState('');

- `const [messages, setMessages] = useState([]);`：定义 `messages` 状态，用于存储聊天记录，初始值为空数组。
- `const [input, setInput] = useState('');`：定义 `input` 状态，用于存储用户输入的消息，初始值为空字符串。

## 发送消息

`sendMessage` 函数负责将用户的消息发送到后端，并使用响应更新状态。

In [ ]:
const sendMessage = async () => {
    if (input.trim() === '') return; // 防止发送空消息
    const response = await axios.post('http://localhost:5000/chat', { message: input });
    setMessages([...messages, { user: 'You', text: input }, { user: 'Bot', text: response.data.answer }]);
    setInput('');
};

- `const sendMessage = async () => {`：定义 `sendMessage` 异步函数，用于发送消息。
- `if (input.trim() === '') return;`：如果输入为空，则不发送消息。
- `const response = await axios.post('http://localhost:5000/chat', { message: input });`：使用 `axios` 发送 POST 请求，将用户输入的消息发送到服务器。
- `setMessages([...messages, { user: 'You', text: input }, { user: 'Bot', text: response.data.answer }]);`：更新 `messages` 状态，添加用户和机器人的消息。
- `setInput('');`：清空输入框。

## 处理按键事件

`handleKeyPress` 函数允许在按下回车键时发送消息。

In [ ]:
const handleKeyPress = (event) => {
    if (event.key === 'Enter') {
        sendMessage();
    }
};

- `const handleKeyPress = (event) => {`：定义 `handleKeyPress` 函数，用于处理键盘按键事件。
- `if (event.key === 'Enter') {`：如果按下的是回车键，则调用 `sendMessage` 函数发送消息。

## 渲染组件

组件渲染聊天界面。

In [ ]:
return (
    <div className="chatbox-container">
        <div className="messages-container">
            {messages.map((msg, index) => (
                <div key={index}><strong>{msg.user}:</strong> {msg.text}</div>
            ))}
        </div>
        <div className="input-container">
            <input
                value={input}
                onChange={(e) => setInput(e.target.value)}
                onKeyPress={handleKeyPress}
            />
            <button onClick={sendMessage}>Send</button>
        </div>
    </div>
);

- `return (`：返回组件的 JSX 结构。
- `<div className="chatbox-container">`：外层容器，包含整个聊天框。
- `<div className="messages-container">`：消息容器，显示所有聊天记录。
- `{messages.map((msg, index) => (`：遍历 `messages` 数组，渲染每条消息。
- `<div key={index}><strong>{msg.user}:</strong> {msg.text}</div>`：显示消息的用户和内容。
- `<div className="input-container">`：输入容器，包含输入框和发送按钮。
- `<input value={input} onChange={(e) => setInput(e.target.value)} onKeyPress={handleKeyPress} />`：输入框，绑定 `input` 状态，处理输入变化和按键事件。
- `<button onClick={sendMessage}>Send</button>`：发送按钮，点击时调用 `sendMessage` 函数。

## 导出组件

最后，导出 `ChatBox` 组件，以便在其他文件中使用。

In [ ]:
export default ChatBox;

# App 组件解释

这个笔记本解释了用 React 实现的 `App` 组件。该组件将 `ChatBox` 组件嵌入到应用程序中。

## 导入所需库

首先，我们导入必要的库和 CSS 文件。

In [ ]:
import React from 'react';
import './App.css';
import ChatBox from './ChatBox';

- `import React from 'react';`：从 `react` 库中导入 `React`。
- `import './App.css';`：导入 `App.css` 文件，用于应用程序的样式。
- `import ChatBox from './ChatBox';`：导入之前定义的 `ChatBox` 组件。

## App 组件定义

`App` 组件被定义为一个函数组件。

In [ ]:
function App() {
  return (
    <div className="App">
      <ChatBox />
    </div>
  );
}

- `function App() {`：定义一个名为 `App` 的函数组件。
- `return (`：返回组件的 JSX 结构。
- `<div className="App">`：外层容器，包含整个应用程序。
- `<ChatBox />`：嵌入 `ChatBox` 组件，显示聊天框。

## 导出组件

最后，导出 `App` 组件，以便在其他文件中使用。

In [ ]:
export default App;

# app.py 模块
本笔记本详细解释了一个基于 Flask 的聊天机器人应用程序，该应用程序集成了 SparkApi。

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import SparkApi
import os
import threading
import time
import re

app = Flask(__name__)
CORS(app)

# 配置环境变量
os.environ["IFLYTEK_SPARK_URL"] = "wss://xingchen-api.cn-huabei-1.xf-yun.com/v1.1/chat"
os.environ["IFLYTEK_SPARK_APP_ID"] = "27741adb"
os.environ["IFLYTEK_SPARK_API_KEY"] = "7bf569c16f9330e40b49fba1ac28383b"
os.environ["IFLYTEK_SPARK_API_SECRET"] = "ZTNhYzdhNzI4NTExMGI4MTJjZjAyNjlk"

# 用于配置大模型版本，默认“general/generalv2”
domain = "xspark13b6k"  # v2.0版本
appid = os.getenv("IFLYTEK_SPARK_APP_ID")
api_secret = os.getenv("IFLYTEK_SPARK_API_SECRET")
api_key = os.getenv("IFLYTEK_SPARK_API_KEY")
Spark_url = os.getenv("IFLYTEK_SPARK_URL")

text = []

def getText(role, content):
    jsoncon = {"role": role, "content": content}
    text.append(jsoncon)
    return text

def getlength(text):
    length = 0
    for content in text:
        length += len(content["content"])
    return length

def checklen(text):
    while getlength(text) > 8000:
        del text[0]
    return text

stored_question = ""
stored_answer = ""

@app.route('/chat', methods=['POST'])
def chat():
    global stored_question, stored_answer
    data = request.get_json()
    stored_question = data.get('message')
    
    # 等待答案生成
    while not stored_answer:
        time.sleep(0.1)
    
    response = stored_answer
    stored_answer = ""  # 重置答案
    return jsonify({'answer': response})

    def process_questions(stored_question, stored_answer, loop=True):
    # global stored_question, stored_answer
    while True:
        if stored_question:
            prompt = ("？。 上面输入的问题属于下面ABCDE的哪一类，请输出对应的字母。\n"
                      "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
                      "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
                      "C: 企业舆情问题。\n"
                      "D: 查询招标信息。\n"
                      "E: 其他 例如:日常生活中常见的问题，通用的问题，不符合招投标领域的。\n")
            
            Input = stored_question + prompt
            stored_answer = ""
            question = checklen(getText("user", Input))
            SparkApi.answer = ""
            print(stored_question)
            print("星火:", end="")
            SparkApi.main(appid, api_key, api_secret, Spark_url, domain, question)
            getText("assistant", SparkApi.answer)
            output = text

            # 提取 'assistant' 的 content 值，并只保留第一个字符
            # assistant_content = next((item['content'] for item in output if item['role'] == 'assistant'), "")
            # stored_answer = assistant_content[0] if assistant_content else ""
            match = re.search(r'[ABCDE]', SparkApi.answer)
            stored_answer = match.group(0) if match else ""
            if stored_answer == "E":
                stored_answer = (" 你的问题描述不够准确，请记得添加关键词。输入的问题应该属于下面ABCD的一类。\n"
                      "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
                      "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
                      "C: 企业舆情问题。\n"
                      "D: 查询招标信息。\n")
            stored_question = ""  # Reset the question after processing
        if loop == False:
            return stored_answer
            break
        time.sleep(1)  # Add a small delay to prevent busy-waiting

if __name__ == '__main__':
    threading.Thread(target=process_questions(stored_question, stored_answer), daemon=True).start()
    app.run(debug=True)




### 导入库
```python
from flask import Flask, request, jsonify
from flask_cors import CORS
import SparkApi
import os
import threading
import time
import re
```
这些行导入了应用程序所需的库：
- 从 `flask` 包中导入 `Flask`、`request` 和 `jsonify` 以创建 Web 应用程序并处理 HTTP 请求和响应。
- 从 `flask_cors` 包中导入 `CORS` 以处理跨域资源共享 (CORS) 问题。
- `SparkApi` 用于与 Spark API 交互。
- `os` 用于处理环境变量。
- `threading`、`time` 和 `re` 分别用于线程、时间管理和正则表达式。

### 初始化 Flask 应用程序
```python
app = Flask(__name__)
CORS(app)
```
这段代码初始化了 Flask 应用程序并为应用程序启用了 CORS。

### 配置环境变量
```python
os.environ["IFLYTEK_SPARK_URL"] = "wss://xingchen-api.cn-huabei-1.xf-yun.com/v1.1/chat"
os.environ["IFLYTEK_SPARK_APP_ID"] = "27741adb"
os.environ["IFLYTEK_SPARK_API_KEY"] = "7bf569c16f9330e40b49fba1ac28383b"
os.environ["IFLYTEK_SPARK_API_SECRET"] = "ZTNhYzdhNzI4NTExMGI4MTJjZjAyNjlk"
```
这些行设置了 Spark API 所需的环境变量，包括 URL、App ID、API Key 和 API Secret。

### 设置模型配置
```python
domain = "xspark13b6k"  # v2.0版本
appid = os.getenv("IFLYTEK_SPARK_APP_ID")
api_secret = os.getenv("IFLYTEK_SPARK_API_SECRET")
api_key = os.getenv("IFLYTEK_SPARK_API_KEY")
Spark_url = os.getenv("IFLYTEK_SPARK_URL")
```
这些行检索环境变量并设置 Spark API 的模型配置。

### 辅助函数
```python
text = []

def getText(role, content):
    jsoncon = {"role": role, "content": content}
    text.append(jsoncon)
    return text

def getlength(text):
    length = 0
    for content in text:
        length += len(content["content"])
    return length

def checklen(text):
    while getlength(text) > 8000:
        del text[0]
    return text
```
这些函数帮助管理文本数据：
- `getText(role, content)`: 使用指定的角色和内容将新消息添加到文本列表中。
- `getlength(text)`: 计算文本列表中所有消息的总长度。
- `checklen(text)`: 确保消息的总长度不超过 8000 个字符，通过删除最旧的消息。

### 全局变量
```python
stored_question = ""
stored_answer = ""
```
这些全局变量存储当前的问题和答案。

### 聊天端点
```python
@app.route('/chat', methods=['POST'])
def chat():
    global stored_question, stored_answer
    data = request.get_json()
    stored_question = data.get('message')
    
    # 等待答案生成
    while not stored_answer:
        time.sleep(0.1)
    
    response = stored_answer
    stored_answer = ""  # 重置答案
    return jsonify({'answer': response})


### 这个端点处理 POST 请求：

- 从请求中获取 JSON 数据并提取消息。
- 等待答案生成。
-  返回生成的答案，并重置存储的答案。

#
#
#





### 这个函数用于处理存储的问题和答案。它接受三个参数：

- stored_question: 存储的问题。
- stored_answer: 存储的答案。
- loop: 一个布尔值，决定是否循环执行。
```python
def process_questions(stored_question, stored_answer, loop=True):



### 这个循环不断检查是否有新的问题需要处理。
```python
while True:
    if stored_question:


### 这段代码生成一个提示，要求用户将输入的问题分类为 A、B、C、D 或 E。
```python
prompt = ("？。 上面输入的问题属于下面ABCDE的哪一类，请输出对应的字母。\n"
          "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
          "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
          "C: 企业舆情问题。\n"
          "D: 查询招标信息。\n"
          "E: 其他 例如:日常生活中常见的问题，通用的问题，不符合招投标领域的。\n")


### 这段代码将用户的问题和提示组合在一起，调用 SparkApi 处理问题，并将答案存储在 text 列表中。
```python
Input = stored_question + prompt
stored_answer = ""
question = checklen(getText("user", Input))
SparkApi.answer = ""
print(stored_question)
print("星火:", end="")
SparkApi.main(appid, api_key, api_secret, Spark_url, domain, question)
getText("assistant", SparkApi.answer)
output = text


### 这段代码使用正则表达式从 SparkApi 的答案中提取分类字母。如果答案是 “E”，则提示用户问题描述不够准确，并要求重新输入。
```python
match = re.search(r'[ABCDE]', SparkApi.answer)
stored_answer = match.group(0) if match else ""
if stored_answer == "E":
    stored_answer = (" 你的问题描述不够准确，请记得添加关键词。输入的问题应该属于下面ABCD的一类。\n"
          "A: 查询企业信息。例如:查询企业的所属城市、营业期限、成立日期、纳税人识别号、法定代表人人、实缴资本、所属行业、匹配状态、所属城市等等一系列和企业信息有关的问题。\n"
          "B: 招投标中的政策、法律咨询相关问题。包括招标投标的一般规定、招标范围、招标程序、监督处理、领域规范。还有政府采购的一般规定、采购主体、政策功能、采购方式等相关问题。\n"
          "C: 企业舆情问题。\n"
          "D: 查询招标信息。\n")


### 处理完问题后，重置 stored_question。
```python
stored_question = ""  # Reset the question after processing


### 如果 loop 为 False，则返回答案并退出循环。否则，添加一个小延迟以防止忙等待。
### 这里是用于测试
```python
if loop == False:
    return stored_answer
    break
time.sleep(1)  # Add a small delay to prevent busy-waiting


### 这段代码启动一个后台线程来处理问题，并运行 Flask 应用程序。
```python
if __name__ == '__main__':
    threading.Thread(target=process_questions(stored_question, stored_answer), daemon=True).start()
    app.run(debug=True)


# test.py模块

## 导入模块
这段代码导入了所需的模块：

- process_questions 和 app 从 app 模块中导入。
- os 用于处理文件路径和环境变量。
- json 用于处理 JSON 文件。
- time 用于时间延迟。
```python
from app import process_questions
import app

import os
import json
import time
`

## 获取文件路径
这段代码获取当前文件的绝对路径和父目录，并构建四个 JSON 文件的路径。
```python
# 获取当前文件的绝对路径
current_file_path = os.path.abspath(__file__)

# 获取当前文件的父目录
parent_directory = os.path.dirname(os.path.dirname(current_file_path))

test_a_path = f"{parent_directory}/variation/a.json"
test_b_path = f"{parent_directory}/variation/b.json"
test_c_path = f"{parent_directory}/variation/c.json"
test_d_path = f"{parent_directory}/variation/d.json"


## 读取 JSON 文件
这段代码打开并读取四个 JSON 文件，将它们的内容加载到 a_lists、b_lists、c_lists 和 d_lists 变量中。
```python 
with open(test_a_path, 'r') as f:
    a_lists = json.load(f)
with open(test_b_path, 'r') as f:
    b_lists = json.load(f)
with open(test_c_path, 'r') as f:
    c_lists = json.load(f)
with open(test_d_path, 'r') as f:
    d_lists = json.load(f)


## 定义 corrent_func 函数
这个函数用于处理问题列表并计算正确答案的数量：

- lists: 问题列表。
- char: 预期的答案字符（A、B、C 或 D）。

函数的主要步骤：

- 初始化 sum 和 count 变量。
- 遍历问题列表，将每个问题传递给 process_questions 函数。
- 如果 stored_answer 与预期的 char 相符，则增加 sum。
- 打印当前的正确答案数量和总问题数量。
- 返回正确答案的总数。
```python
def corrent_func(lists, char):
    sum = 0
    count = 0
    stored_question = app.stored_question
    stored_answer = app.stored_answer
    for list in lists:
        stored_question = list[0]
        stored_answer = process_questions(stored_question, stored_answer, loop=False)
        time.sleep(2)
        if stored_answer == char:
            sum += 1
        count += 1
        print(f"{char}_corrent:", sum, "\n")
        print(f"{char}_count:", count, "\n")
        
    return sum


## 计算正确答案数量
这段代码分别计算四类问题（A、B、C、D）的正确答案数量，并打印结果。每次计算后，程序会暂停 30 秒。
```python
sum_a = corrent_func(a_lists, "A")
time.sleep(30)
sum_b = corrent_func(b_lists, "B")
time.sleep(30)
sum_c = corrent_func(c_lists, "C")
time.sleep(30)
sum_d = corrent_func(d_lists, "D")
print(" correct number of class A questions --- %d ---  per 100 \n ", sum_a)
print(" correct number of class B questions --- %d ---  per 100 \n ", sum_b)
print(" correct number of class C questions --- %d ---  per 100 \n ", sum_c)
print(" correct number of class D questions --- %d ---  per 100 \n ", sum_d)
